In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# NOTE: ONLY NEEDED FOR GOOGLE COLLAB
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [10]:
# NOTE: ONLY for google drive and may be a different root directory and may not be necessary 
%cd '/content/drive/My Drive/EQWorksInterview'
!ls

[WinError 3] The system cannot find the path specified: "'/content/drive/My Drive/EQWorksInterview'"
C:\Users\tahas\Downloads\EQWorksInterview\EQWorksInterview


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
# Taking in data
dataset = pd.read_csv('DataSample.csv')
poi_dataset = pd.read_csv('POIList.csv')
print((poi_dataset))

  POIID   Latitude   Longitude
0  POI1  53.546167 -113.485734
1  POI2  53.546167 -113.485734
2  POI3  45.521629  -73.566024
3  POI4  45.224830  -63.232729


In [15]:
# Removing "suspicious" data
dataset = dataset.drop_duplicates(subset=['Latitude', 'Longitude', ' TimeSt'], keep=False)

In [17]:
import math
from geopy import distance

# Used to return geographic distsance between two coordinates
def getDistance(lat1, lon1, lat2, lon2, unit):
    coor1 = (lat1, lon1)
    coor2 = (lat2, lon2)

    return distance.distance(coor1, coor2).km

# Used to return the closest POI for a given coordinate
def returnPOI(arr):
  lat1 = arr[0]
  lon1 = arr[1]

  distances_to_POI = {}
  for index, row in poi_dataset.iterrows():
    id = (row["POIID"]).replace(" ","")
    curr_latitude = (row[" Latitude"])
    curr_longitude = (row["Longitude"])
    distance_to_poi = getDistance(lat1, lon1, curr_latitude, curr_longitude, "K")
    distances_to_POI[id] = distance_to_poi
  
  min_key = (min(distances_to_POI, key=distances_to_POI.get))
  return (distances_to_POI[min_key], min_key)

In [18]:
# Calculating the closest POI for each request

x=dataset.iloc[:,5:7].values
POIlis = []
distances = []
for element in x:
  (currDistance,shortestPOI) = returnPOI(element)
  POIlis.append(shortestPOI)
  distances.append(currDistance)


In [19]:
# Assigning each request the closest POI and distance to the POI
dataset.insert(7, "POI", POIlis )
dataset.insert(8, "distance", distances )

In [20]:
# Organizing requests corresponding to their POI group 
DataFrames_dict = {}
for index,row in poi_dataset.iterrows():
  curr_poi = row["POIID"]
  currDataFrame = dataset[dataset.POI == curr_poi]
  DataFrames_dict[curr_poi] = currDataFrame


In [21]:
# Calculating the mean of all distances from request to POI for every POI
Means_dict={}
for POI, dataframe in DataFrames_dict.items():
  if np.isnan(dataframe["distance"].mean()):
    Means_dict[POI] = 0
  else:
    Means_dict[POI] = dataframe["distance"].mean()

print(Means_dict)

{'POI1': 301.1041430390966, 'POI2': 0, 'POI3': 452.79513736084493, 'POI4': 516.1163246993544}


In [22]:
# Calculating the standard deviation of distance data for every POI 
# Standard deviations
Standard_deviation_dict = {}
for POI, dataframe in DataFrames_dict.items():
  Standard_deviation_dict[POI] = dataframe["distance"].std()

print(Standard_deviation_dict)

{'POI1': 388.80401983377766, 'POI2': nan, 'POI3': 224.4312193678686, 'POI4': 1510.305291650465}


In [23]:
# Calculating radius of every POI if we map it into a circle
Radius_dict = {}
for POI, dataframe in DataFrames_dict.items():
  if not dataframe.empty:
    Radius_dict[POI] = max( dataframe["distance"] )
    # print(max( dataframe["distance"] ))
  else:
    Radius_dict[POI] = 0
  # print(dataframe["distance"])

print(Radius_dict)

{'POI1': 11541.82947035032, 'POI2': 0, 'POI3': 1500.453242009817, 'POI4': 9365.335692650428}


In [24]:
# Calculating the density of each POI using (requests/area)
Density_dict = {}
for POI, dataframe in DataFrames_dict.items():
  if Radius_dict[POI] != 0:
    Density_dict[POI] = len(dataframe["distance"]) / ( math.pi * Radius_dict[POI]**2 )
  else:
    Density_dict[POI] = 0

print(Density_dict)

{'POI1': 2.090304689925981e-05, 'POI2': 0, 'POI3': 0.0012446174816043488, 'POI4': 1.531496047948358e-06}


In [25]:
# Getting the total number of requests for each POI using 
Total_Requests_dict = {}
for POI, dataframe in DataFrames_dict.items():
  if Radius_dict[POI] != 0:
    Total_Requests_dict[POI] = len(dataframe["distance"])
  else:
    Total_Requests_dict[POI] = len(dataframe["distance"])

print(Total_Requests_dict)

{'POI1': 8748, 'POI2': 0, 'POI3': 8803, 'POI4': 422}


In [26]:
# 4a) Modeling Popularity

In [27]:
# Calculating the standardized number of requests for each POI between 0 and 1
standardized_total_requests_dict = {}

max_requests = Total_Requests_dict[max(Total_Requests_dict, key=Total_Requests_dict.get)]
min_requests = Total_Requests_dict[min(Total_Requests_dict, key=Total_Requests_dict.get)]

for POI, total_requests in Total_Requests_dict.items():
  zi = ( total_requests - min_requests ) / (max_requests - min_requests)
  # # transforming to become -10 and +10
  # zi = (zi - 0.5) * 20
  if not (zi == None):
    standardized_total_requests_dict[POI] = zi
  
print(standardized_total_requests_dict)

{'POI1': 0.993752129955697, 'POI2': 0.0, 'POI3': 1.0, 'POI4': 0.047938202885379984}


In [28]:
# Calculating the standardized average distance between 0 and 1 for every POI in dataset
standardized_average_distances_dict = {}

max_mean = Means_dict[max(Means_dict, key=Means_dict.get)]
min_mean = Means_dict[min(Means_dict, key = Means_dict.get)]

for POI, mean in Means_dict.items():
  zi = ( mean - min_mean ) / (max_mean - min_mean)
  # # transforming to become -10 and +10
  # zi = (zi - 0.5) * 20
  if not (zi == None):
    standardized_average_distances_dict[POI] = zi
  
print(standardized_average_distances_dict)

#{'POI1': 0.0, 'POI2': nan, 'POI3': 0.7054995356562442, 'POI4': 1.0}

{'POI1': 0.5834036410580392, 'POI2': 0.0, 'POI3': 0.8773121788476753, 'POI4': 1.0}


In [29]:
# standardized_distances

In [30]:
# Getting a standardixed average density between 0 and 1 for every POI in dataset
standardized_densities_dict = {}

max_density = Density_dict[max(Density_dict, key=Density_dict.get)]
min_density = Density_dict[min(Density_dict, key = Density_dict.get)]

for POI, density in Density_dict.items():
  zi = ( density - min_density ) / (max_density - min_density)
  # transforming to become -10 and +10
  # zi = (zi - 0.5) * 20
  standardized_densities_dict[POI] = zi
print(standardized_densities_dict)

{'POI1': 0.01679475598584327, 'POI2': 0.0, 'POI3': 1.0, 'POI4': 0.001230495369528487}


In [31]:
# Calculating the relative popularity score for each POI and storing it in the 'popularities' dict
standardized_distances = list(standardized_average_distances_dict.values())
standardized_densities = list(standardized_densities_dict.values())
standardized_total_requests = list(standardized_total_requests_dict.values())

final_df = pd.DataFrame({"distances": standardized_distances, "densities": standardized_densities, "total_requests":standardized_total_requests})
new_popularities = {}
for index, row in final_df.iterrows():
  # print(index, " ", row['distances'], " ", row['densities'], row["total_requests"])
  
  # transforming to become -10 and +10
  distance_score = ( (1- row['distances']) - 0.5) * 1.5
  density_score = (row['densities'] - 0.5) * 6.3
  requests_score = (row['total_requests'] - 0.5) * 12.2

  new_popularity_score = distance_score + density_score + requests_score
  new_popularities[ f"POI{index+1}" ] = new_popularity_score

In [32]:
print("new:",new_popularities)
# Note: the popularity values are a bit skewed on both ends but that's due to the
# there only bening 4 POI's. As this data gets scaled, it will be a much better
# indicator of 'relative popularity' and will be less skewed.

new: {'POI1': 2.854477486583256, 'POI2': -8.5, 'POI3': 8.684031731728487, 'POI4': -9.407401803970334}
